In [2]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize

In [3]:
CLIENT_ID = 'YWHRWZENBRFM4LEANKM4VJERTJWP3I5G3A1340YMNFQKE5ON' # your Foursquare ID
CLIENT_SECRET = 'CO43MLZ4D20PPPJQTJRA0RMHCQCVAYOKQWZMLW1B2BZD1ZK0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YWHRWZENBRFM4LEANKM4VJERTJWP3I5G3A1340YMNFQKE5ON
CLIENT_SECRET:CO43MLZ4D20PPPJQTJRA0RMHCQCVAYOKQWZMLW1B2BZD1ZK0


In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

latitude=44.16975536259105 
longitude=9.88543293779643

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=YWHRWZENBRFM4LEANKM4VJERTJWP3I5G3A1340YMNFQKE5ON&client_secret=CO43MLZ4D20PPPJQTJRA0RMHCQCVAYOKQWZMLW1B2BZD1ZK0&v=20180605&ll=44.16975536259105,9.88543293779643&radius=5000&limit=100'

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Osteria Delle Pietre,Pizza Place,44.142197,9.888887
1,Il Ristoro nell'Aia,Italian Restaurant,44.157475,9.903568
2,Locanda La Trigola,Restaurant,44.160703,9.934623
3,Decathlon,Sporting Goods Shop,44.153091,9.917256
4,OVS,Clothing Store,44.155107,9.917197
5,L'Ombrosa,Italian Restaurant,44.155823,9.878426
6,Centro Commerciale La Fabbrica / E.Leclerc,Shopping Mall,44.155188,9.917136
7,Stazione Santo Stefano di Magra,Train Station,44.155960,9.915837
8,Conad,Supermarket,44.155293,9.916929
9,Bricofer,Hobby Shop,44.135553,9.902525
